# Is the NN enough to identify LRU or LFU indices?

In this notebook I check if the nn for my DQN even capable of doing its task?

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import gym
import time
from e2 import CacheEnv
from tqdm import tqdm_notebook as tqdm

In [2]:
# env vars
EPS_LEN = 100
N_PAGES = 20
CACHE_LIMIT = 10 
env = CacheEnv(
        eps_len=EPS_LEN, 
        n_pages=N_PAGES, 
        limit=CACHE_LIMIT
        )

# dqn vars
# N_EPS = 60000
N_EPS = 1000
BATCH_SIZE = 32
LR_adam = 3e-4                   # learning rate for Adam
LR_sgd = 1e-3                   # learning rate for SGD
EPSILON = 0.9               # greedy policy
GAMMA = 0.9                 # reward discount
TARGET_REPLACE_ITER = 2000   # target update frequency
MEMORY_CAPACITY = 20000

s = env.reset()
N_ACTIONS = env.action_space_n
STATE_SHAPE = (CACHE_LIMIT, 2)
N_STATES = STATE_SHAPE[0]*STATE_SHAPE[1]

Cache limit: 10
Total Pages: 20


In [3]:
# Collect data
dataX = []
for _ in tqdm(range(N_EPS)):
    s = env.reset()
    dataX.append(s)

    while True:
        a = np.random.randint(0, N_ACTIONS)
        s, _, done, _ = env.step(a)
        dataX.append(s)
        
        if done:
            break

print(len(dataX))            


101000


In [4]:
def get_labels(dataX):
    dataYLU = []
    dataYRU = [] 
    for x in tqdm(dataX):
        lus = np.argmin(x[::2])
        rus = np.argmin(x[1::2])
        dataYLU.append(lus)
        dataYRU.append(rus)
    return dataYLU, dataYRU     
        
dataYLU, dataYRU = get_labels(dataX)

In [5]:
print(len(dataX))
print(len(dataYLU))
print(len(dataYRU))

101000
101000
101000


In [6]:
from sklearn.model_selection import train_test_split
X = dataX
Y = dataYLU
XTrain, XTest, yTrain, yTest = train_test_split(X, Y, test_size = 0.2)
print(len(X))
print(len(Y))
print(len(XTrain))
print(len(XTest))

101000
101000
80800
20200


In [7]:
from torch.utils.data import Dataset, DataLoader

class CacheDataset(Dataset):
    def __init__(self, data, targets, transform=None):
        self.transform = transform
        self.data = torch.Tensor(data)
        self.targets = torch.LongTensor(targets)

    def __getitem__(self, index):
        x = self.data[index]
        y = self.targets[index]
        return x, y

    def __len__(self):
        return self.data.shape[0]

In [8]:
train_dataset = CacheDataset(XTrain, yTrain)
test_dataset = CacheDataset(XTest, yTest)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

In [16]:
class Net(nn.Module):
    def __init__(self, ):
        super(Net, self).__init__()
        input_size = N_STATES
        h_dim = 50
        self.fc1 = nn.Linear(input_size, h_dim)
        self.bn1 = nn.BatchNorm1d(h_dim)
        self.fc2 = nn.Linear(h_dim, h_dim//4)
        self.bn2 = nn.BatchNorm1d(h_dim//4)
        self.fc3 = nn.Linear(h_dim//4, h_dim)
        self.bn3 = nn.BatchNorm1d(h_dim)
        self.out = nn.Linear(h_dim, N_ACTIONS)

    def forward(self, x):
#         bs = x.shape[0]
        x = F.relu(self.fc1(x))
        x = self.bn1(x)
        x = F.relu(self.fc2(x))
        x = self.bn2(x)
        x = F.relu(self.fc3(x))
        x = self.bn3(x)
        x = self.out(x)
#         return x
        return F.softmax(x, dim=0)    

In [17]:
model = Net().cuda()
LR_adam = 3e-3
optimizer = torch.optim.Adam(model.parameters(), lr=LR_adam)
# optimizer = torch.optim.SGD(model.parameters(), lr=LR_sgd)
# loss_func = nn.MSELoss()
criterion = nn.CrossEntropyLoss()

epochs = 100
for epoch in tqdm(range(epochs)):
    totalloss = []
    for i, (X, y) in enumerate(train_loader):
        X, y = X.cuda(), y.cuda()
        optimizer.zero_grad()
        out = model(X)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        with torch.no_grad():
            totalloss.append(loss.detach().cpu().numpy())
    totalloss = np.array(totalloss).mean()
    print(f"Epoch: [{epoch}] | Loss: {totalloss}")               

Epoch: [0] | Loss: 2.187633514404297
Epoch: [1] | Loss: 2.144232749938965
Epoch: [2] | Loss: 2.1371021270751953
Epoch: [3] | Loss: 2.1437060832977295
Epoch: [4] | Loss: 2.121243715286255
Epoch: [5] | Loss: 2.1124751567840576
Epoch: [6] | Loss: 2.105055093765259
Epoch: [7] | Loss: 2.0993552207946777
Epoch: [8] | Loss: 2.0964062213897705
Epoch: [9] | Loss: 2.0916545391082764
Epoch: [10] | Loss: 2.0877633094787598
Epoch: [11] | Loss: 2.085021734237671
Epoch: [12] | Loss: 2.0823757648468018
Epoch: [13] | Loss: 2.0809569358825684
Epoch: [14] | Loss: 2.0783753395080566
Epoch: [15] | Loss: 2.075209140777588
Epoch: [16] | Loss: 2.0703678131103516
Epoch: [17] | Loss: 2.0702061653137207
Epoch: [18] | Loss: 2.066880702972412
Epoch: [19] | Loss: 2.0660462379455566
Epoch: [20] | Loss: 2.0639312267303467
Epoch: [21] | Loss: 2.061947822570801
Epoch: [22] | Loss: 2.060666561126709
Epoch: [23] | Loss: 2.059048891067505
Epoch: [24] | Loss: 2.057486057281494
Epoch: [25] | Loss: 2.0560691356658936
Epoch: 

In [ ]:
model = Net().cuda()
LR_adam = 3e-3
optimizer = torch.optim.Adam(model.parameters(), lr=LR_adam)
# LR_sgd = 1
# optimizer = torch.optim.SGD(model.parameters(), lr=LR_sgd)
# loss_func = nn.MSELoss()
criterion = nn.CrossEntropyLoss()

epochs = 100
for epoch in tqdm(range(epochs)):
    totalloss = []
    for i, (X, y) in enumerate(train_loader):
        X, y = X.cuda(), y.cuda()
        optimizer.zero_grad()
        out = model(X)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        with torch.no_grad():
            totalloss.append(loss.detach().cpu().numpy())
    totalloss = np.array(totalloss).mean()
    print(f"Epoch: [{epoch}] | Loss: {totalloss}")               